In [70]:
# import
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Categorical
from itertools import repeat

import numpy as np
import pandas as pd
from Utils import custom_data_loader, preprocess_data
from Utils.SummaryWriter import LogSummary
from Models.simpleFFBNN import SimpleFFBNN
from Models.denseRegression import DenseRegressor

from torch.utils.data import DataLoader
from torch.utils.data import SubsetRandomSampler

from sklearn.metrics import r2_score
import os

In [2]:
def get_device():
    """Function to get the device to be used for training the model
    """
    cuda = torch.cuda.is_available()
    print("CUDA Available: ", cuda)

    if cuda:
        gpu = GPUtil.getFirstAvailable()
        print("GPU Available: ", gpu)
        torch.cuda.set_device(gpu)
        device = torch.device('cuda')
    else:
        device = torch.device('cpu')
    print("Device: ", device)
    return device

device = get_device()


CUDA Available:  False
Device:  cpu


In [9]:
# load data
dataloader_train, dataloader_test, dataloader_val = preprocess_data(pd.read_csv('/Users/kristian/Documents/Skole/9. Semester/Thesis Preparation/Code/BNNs/Data/quality_of_food.csv'), batch_size = 128)


/Users/kristian/Documents/Skole/9. Semester/Thesis Preparation/Code/BNNs/Utils/config.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.X['savings'] = np.where(self.X['savings'] == 'low', 0, np.where(self.X['savings'] == 'medium', 1, 2))


In [3]:
model = SimpleFFBNN(4, 1)

In [8]:
checkpoint = torch.load('/Users/kristian/Documents/Skole/9. Semester/Thesis Preparation/Code/BNNs/trainedModels/simple_model.pth', map_location=torch.device('cpu'))
#print(checkpoint)
#model.load_state_dict(checkpoint['model'])
model.load_state_dict(checkpoint)
    

<All keys matched successfully>

In [4]:
class Saveoutput():
    def __init__(self, instances, batch_size, rounds):
        self.T = instances
        self.batch_size = batch_size
        self.outputs = []
        self.rounds = rounds
        self.counter = 0


    def __call__(self, module, module_in, module_out):
        if self.counter < 3:
            sample_data = np.random.randint(self.batch_size)
            outs = module_out.view(self.batch_size, -1)
            #outs = module_out.view(self.T, self.batch_size, -1)[:, 0, :]
            layer_size = outs.shape[1]

            
            write_summary.per_round_layer_output(layer_size, outs, self.rounds)

            self.counter += 1


    def clear(self):
        self.outputs = []
        


        

In [4]:
write_summary = LogSummary(name = 'Simple Model')

In [132]:
class runActiveLearning():
    def __init__(self, model_name, model, dataloader_train, dataloader_test, dataloader_val, epochs, rounds, learning_rate, batch_size, instances, highest_unc, seed_sample, retrain, resume_round):
        self.model_name = model_name
        self.model = model
        self.dataloader_train = dataloader_train
        self.dataloader_test = dataloader_test
        self.dataloader_val = dataloader_val
        self.epochs = epochs
        self.rounds = rounds
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.instances = instances
        self.highest_unc = highest_unc
        self.seed_sample = seed_sample


        # a set of lists to store the selected indices with highest uncertainty
        self.highest_unc = set([])
        # unexplored data
        self.unexplored_data = set(range(len(dataloader_train.dataset)))

        if resume_round and not retrain:
            self.InitModel(load_weigths=True, res_round=resume_round)
        elif resume_round and retrain:
            self.InitModel(load_weigths=False, res_round=resume_round)
        else:
            self.InitModel()


        training_params = sum(p.numel() for p in self.model.parameters())
        print('Running Model:{}'.format(model, training_params, self.epochs, self.batch_size))


    def InitModel(self,load_weigths = False, res_round = None):
        if self.model_name == 'simple':
            self.model = SimpleFFBNN(4, 1, )
        else:
            self.model = DenseRegressor(4, 1)
            
    def loadSeedModel(self):
        # load the trained model
        model = SimpleFFBNN(4, 1)
        checkpoint = torch.load('Activelearning/SeedModels/simple_model_best.pth', map_location=torch.device('cpu'))
        model.load_state_dict(checkpoint)
        self.model = model
        print(f'Model loaded: {model}')

    def loadPreTrainedModel(self, load_weigths, res_round):
        self.getTrainedModel(res_round)
        # load weights
        if DEVICE.type == 'cpu':
            state = torch.load(self.train_weight_path, map_location=torch.device('cpu'))
        else:
            state = torch.load(self.train_weight_path)

        self.highest_unc = state['highest_unc']
        optimizer = torch.optim.Adam(self.model.parameters(), lr=self.learning_rate)

        if load_weigths:
            self.model.load_state_dict(state['weights'])
            self.optimizer.load_state_dict(state['optimizer'])

    def getEntropy(self, y):
        ensemble_probabilities = y.mean(0)
       # print(f'Ensemble probabilities func: {ensemble_probabilities.shape}')
        entropy = Categorical(probs = ensemble_probabilities).entropy()
        # get the entropy of the ensemble probabilities
        #entropy = -torch.sum(ensemble_probabilities * torch.log(ensemble_probabilities), dim=1)

        #probabilities = Categorical(probs = ensemble_probabilities)

       # print(f'Probabilities func: {probabilities.probs[0:5]}')

        #entropy = probabilities.entropy()

        #entropy  = -torch.sum(ensemble_probabilities * torch.log(ensemble_probabilities), dim=1)

        #print(f'Entropy func: {entropy[0:5]}')
        return entropy


    def trainSeedModelClosedForm(self, rounds):
        # loss function
        criterion = nn.MSELoss()
        optimizer = torch.optim.Adam(self.model.parameters(), lr=self.learning_rate)


        if rounds == 1:
            self.highest_unc = set(range(self.seed_sample))
            self.unexplored_data = self.unexplored.difference(self.highest_unc)



        uncertainty = []
        hook_handles = []
        save_output = Saveoutput(instances= self.instances, batch_size=self.batch_size, rounds = self.rounds)
        self.model.eval()
        for layer in self.model.kl_layers:
            hook_handles.append(layer.register_forward_hook(save_output))

        with torch.no_grad():
            for batch_index, (X, y) in enumerate(self.dataloader_train):
                batch_size = X.shape[0]
                #print(f'X before repeat: {X}')
               # print(f'y before repeat: {y}')
                save_output.batch_size = batch_size
                X = X.repeat(self.instances, 1)
                #print(f'X after repeat: {X}, {X.shape}')
                

                y = y.squeeze()
                # squeeze the y to remove the extra dimension
                
                y = y.repeat(self.instances)
        
                X, y = X.to(device), y.to(device)
                y_pred = model(X)
                print(f'y_pred: {y_pred.shape}, {y_pred[0:5]}, {y_pred[128:133]}')
                
                

                ensemble_outputs = y.reshape(self.instances, batch_size, 1) # reshape to instances x batch_size x 1 (output size)
                
                
                uncertainty.append(self.getEntropy(ensemble_outputs))
                   
        
            save_output.clear()
            save_output.counter = 0
            for handle in hook_handles:
                handle.remove()
            
            uncertainty = torch.cat(uncertainty)
            
            new_indices = torch.argsort(uncertainty, descending=True).tolist()

            # remove already selected data
            new_indices = [i for i in new_indices if i not in self.highest_unc]  

            # select the highest uncertainty data
            self.highest_unc.union(set(new_indices[:self.seed_sample]))
            print(f'higheset_unc: {self.highest_unc}')
            self.unexplored_data = self.unexplored_data.difference(self.highest_unc)

    
    def objective(self, output, target, kl, beta):
        loss_fun = nn.MSELoss()
        discrimination_error = loss_fun(output.view(-1), target.view(-1))
        variational_bound = discrimination_error + beta * kl
        return variational_bound, discrimination_error, kl


    def getTrainedModel(self, res_round):
        retrain = self.retrain
        self.train_weight_path = f'Activelearning/SeedModels/simple_model_best.pth'
        if res_round:
            self.train_weight_path = f'Activelearning/SeedModels/simple_model_best.pth'
        else:
            self.train_weight_path = f'Activelearning/SeedModels/simple_model_best.pth'

        return (self.model, self.train_weight_path)



    def trainModel(self, rounds, epochs):
        t_total, v_total = 0, 0
        t_r2_scores, v_r2_scores = [], []
        self.model.train()
        t_loss, v_loss = [], []
        t_likelihood, v_likelihood = [], []
        t_kl, v_kl = [], []
        self.model.train()
        m = len(self.dataloader_train)

        for batch_index, (inputs, targets) in enumerate(self.dataloader_train):
            X = inputs.repeat(1, 1)  # (number of mcmc samples)
            Y = targets.repeat(1)
            X, Y = X.to(device), Y.to(device)
            outputs = self.model(X)
            loss, log_likelihood, kl = self.objective(outputs, Y, self.model.kl_divergence, m)
        
        t_total += target.size(0)
        t_r2_scores.append(r2_score(outputs.detach().cpu()[:,0], targets.numpy()))
        t_loss.append(loss.item())
        loss.backward()
        optimizer.step()
        for layer in self.model.kl_layers:
            layer.clip_variances()

        
        # validate
        m_val = len(self.dataloader_val)
        self.model.eval()
        for batch_index, (inputs, targets) in enumerate(self.dataloader_val):
            X = self.model(inputs)
            loss_val, log_likelihood_val, kl_val = self.objective(outputs, Y, self.model.kl_divergence, m_val)
        
            v_total += target.size(0)
            v_loss.append(loss_val.item())
            v_likelihood.append(log_likelihood_val.item())
            v_kl.append(kl_val.item())
           # v_r2_scores.append(r2_score(outputs.detach().cpu()[:,0], targets.numpy()))
            


            

   # write_summary = LogSummary(name='Simple Model')





# use the class to run the active learning
active_learning = runActiveLearning(model_name='simple', model=model, dataloader_train=dataloader_train, dataloader_test=dataloader_test, dataloader_val=dataloader_val, epochs=100, rounds=2, learning_rate=0.001, batch_size=128, instances=3, highest_unc=6, seed_sample=4, retrain=False, resume_round=False)

Running Model:SimpleFFBNN(
  (fc1): KlLayers (4 -> 10)
  (fc2): KlLayers (10 -> 20)
  (fc3): KlLayers (20 -> 1)
)


In [8]:
def trainSeedModelClosedForm(self):
    hook_handles = []
    save_output = saveOutput(self.instances, self.batch_size, 0)
    self.model.eval()
    for layer in self.model.layers:
        hook_handles.append(layer.register_forward_hook(save_output))
    
    with torch.no_grad():
        for batch_index, (X, y) in enumerate(self.data_train):
            batch_size = X.shape[0]
            save_output.batch_size = batch_size
            X = X.repeat(self.instances, 1)
            y = y.repeat(self.instances)
            X, y = X.to(self.device), y.to(self.device)
            ensemble_outputs = self.model(X)

        save_output.clear()
        save_output.counter = 0
        for handle in hook_handles:
            handle.remove()


In [5]:
# load data
dataloader_train, dataloader_test, dataloader_val, dataset_train, dataset_test, dataset_val = preprocess_data(pd.read_csv('/Users/kristian/Documents/Skole/9. Semester/Thesis Preparation/Code/BNNs/Data/quality_of_food.csv'), batch_size = 128)

/Users/kristian/Documents/Skole/9. Semester/Thesis Preparation/Code/BNNs/Utils/config.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.X['savings'] = np.where(self.X['savings'] == 'low', 0, np.where(self.X['savings'] == 'medium', 1, 2))


In [105]:
class runActiveLearning():
    def __init__(self, model_name, model, dataloader_train, dataloader_test, dataloader_val, epochs, rounds, learning_rate, 
    batch_size, instances, seed_sample, retrain, resume_round, optimizer):
        self.model_name = model_name
        self.model = model
        self.dataloader_train = dataloader_train
        self.dataloader_test = dataloader_test
        self.dataloader_val = dataloader_val
        self.epochs = epochs
        self.rounds = rounds
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.instances = instances
        self.seed_sample = seed_sample
        self.optimizer = optimizer

        # a set of lists to store the selected indices with highest uncertainty
        self.selected_data = set([])
        # unexplored data
        self.unexplored_data = set(range(len(dataloader_train)))

        # make sure sklearn.metrics.r2_score is imported
        #self.r2_score = r2_score



    
    def objective(self, output, target, kl, beta):
        '''Objective function to calculate the loss function / KL divergence'''
        loss_fun = nn.MSELoss()
        discrimination_error = loss_fun(output.view(-1), target)
        variational_bound = discrimination_error + beta * kl
        return variational_bound, discrimination_error, kl



    def get_validation_data(self, is_validation):
        if not is_validation:
            # train sampler randomly samples data from the selected data set
            train_sampler = SubsetRandomSampler(list(self.selected_data))
            # train loader will load the data from the train sampler
            self.train_loader = DataLoader(self.dataloader_train, batch_size=self.batch_size, sampler=train_sampler, num_workers=1)

        indices = list(self.unexplored_data)
        np.random.shuffle(indices)
        split = int(np.floor(0.1 * len(indices)))  # this line is to split the data into 90% training and 10% validation
        validation_idx = np.random.choice(indices, size = split) # this line is to randomly select 10% of the data for validation
        train_sampler = SubsetRandomSampler(list(self.selected_data))
        validation_sampler = SubsetRandomSampler(validation_idx)
        self.train_loader = DataLoader(self.dataloader_train, batch_size=self.batch_size, sampler=train_sampler, num_workers=1)
        self.validation_loader = DataLoader(self.dataloader_train, batch_size=self.batch_size, sampler=validation_sampler, num_workers=1)

    def random_data(self, rounds):
        # randomly select data
        self.selected_data = set(range(self.seed_sample))
        self.unexplored_data = self.unexplored_data.difference(self.selected_data)
        

    def TrainModel(self, rounds, epochs, is_validation):
        print('running model')
        t_total, v_total = 0, 0
        t_r2_scores = []
        if epochs == 1:
            self.get_validation_data(is_validation)
        self.model.train()
        t_loss, v_loss = [], []
        t_likelihood, v_likelihood = [], []
        t_kl, v_kl = [], []
        self.model.train()
        m = len(self.train_loader)

        print('before loop, this is the train loader: {}'.format(self.train_loader), len(self.train_loader))
        for batch_index, (inputs, targets) in enumerate(self.train_loader):
            print('running loop')
            X = inputs.repeat(1, 1) # (number of mcmc samples, input size)
            Y = targets.repeat(1, 1)
            X, Y = X.to(device), Y.to(device)
            outputs = self.model(X)
            loss, log_likelihood, kl = self.objective(outputs, Y, self.model.kl_divergence(), 1 / m)
            t_likelihood.append(log_likelihood.item())
            t_kl.append(kl.item())
            t_total += targets.size(0)
          
            # calculate r2 score manually
            r2_score_value = 1 - (np.sum((outputs.detach().cpu().numpy() - targets.detach().cpu().numpy()) ** 2) / np.sum((targets.detach().cpu().numpy() - np.mean(targets.detach().cpu().numpy())) ** 2))
            t_r2_scores.append(r2_score_value)
            
            t_loss.append(loss.item())
            loss.backward()

            # define the optimizer
            optimizer = self.optimizer

            optimizer.step()
            for layer in self.model.kl_layers:
                layer.clip_variances()
        
        if is_validation:
            print(f'this is the validation data {self.validation_loader}, these are the characteristics {len(self.validation_loader)}')
            m_val = len(self.validation_loader)
            self.model.eval()
            for batch_index, (inputs, targets) in enumerate(self.validation_loader):
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = self.model(inputs)
                loss_val, log_likelihood_val, kl_val = self.objective(outputs, targets, self.model.kl_divergence(), 1 / m_val)
                v_total += targets.size(0)
                v_loss.append(loss_val.item())
                v_likelihood.append(log_likelihood_val.item())
                v_kl.append(kl_val.item())

            
            avg_v_loss = np.average(v_loss)
            avg_t_loss = np.average(t_loss)
            avg_v_likelihood = np.average(v_likelihood)
            avg_t_likelihood = np.average(t_likelihood)
            avg_v_kl = np.average(v_kl)
            avg_t_kl = np.average(t_kl)


            print(
                'epochs: {}, train loss: {}, train likelihood: {}, train kl: {}'.format(
                    epochs, avg_t_loss, \
                    avg_t_likelihood, avg_t_kl))

            print(
                'epochs: {}, validation loss: {}, validation likelihood: {}, validation kl: {}'.format(
                    epochs, avg_v_loss, \
                    avg_v_likelihood, avg_v_kl))

            return avg_v_loss

        else:
            avg_t_loss = np.average(t_loss)
            avg_t_likelihood = np.average(t_likelihood)
            avg_t_kl = np.average(t_kl)
            avg_t_r2 = np.average(t_r2_scores)

            print(
                'epochs: {}, train loss: {}, train likelihood: {}, train kl: {}, train_avg_R2: {}'.format(
                    epochs, avg_t_loss, \
                    avg_t_likelihood, avg_t_kl, avg_t_r2))

            return avg_t_loss, avg_t_r2

    
    def TestModel(self, rounds):
        if device.type == 'cpu':
            state = torch.load(self.train_weight_path, map_location=torch.device('cpu'))
        else:
            state = torch.load(self.train_weight_path)

        self.model.load_state_dict(state['weights'])
        print(f'Model loaded: {self.model}')

        self.model.eval()
        predictions = []
        actual = []
        mse_scores = []
        with torch.no_grad():
            for batch_index, (inputs, targets) in enumerate(self.dataloader_test):
                X, Y = inputs.to(device), targets.to(device)
                outputs = self.model(inputs)

                X = X.detach().cpu().numpy()
                predictions.append(X)
               
                actual.append(Y.detach().cpu().numpy())
                
                mse_scores.append(nn.MSELoss(outputs, targets).item())
        print(f'MSE scores: {mse_scores}')
        



    def getTrainedModel(self, rounds):
        # path to save the trained model
        self.train_weight_path = 'trainedModels/trained_weights/' + self.model_name + '_' + 'e' + str(self.epochs) + '_' + '-r' + str(rounds) + '-b' + str(self.batch_size) + '.pkl'
        return (self.model, self.train_weight_path)


    def saveModel(self, model, optimizer, path_to_save):
        state = {
            'rounds': self.rounds,
            'weights': model.state_dict(),
            'selected_data': self.selected_data,
            'optimizer': self.optimizer.state_dict()
            }

        path_to_save = 'trainedModels/trained_weights/' + self.model_name + '_' + 'e' + str(self.epochs) + '_' + '-r' + str(self.rounds) + '-b' + str(self.batch_size) + '.pkl'

        torch.save(state, path_to_save)
        
            


            


    

        
        


        

In [106]:
if not os.path.isdir('trainedModels/trained_weights'):
    os.makedirs('trainedModels/trained_weights')


# use the class to run the active learning
active_learning = runActiveLearning(model_name='simple', model=model, dataloader_train=dataset_train, dataloader_test=dataset_test, dataloader_val=dataset_val, epochs=100, rounds=2, learning_rate=0.001, batch_size=128, instances=3, seed_sample=4, retrain=False, resume_round=False, optimizer= torch.optim.Adam(model.parameters(), lr=0.001))

active_learning.get_validation_data(is_validation = True)

active_learning.random_data(rounds = 2)

for e in range(5):
    # if is_validation is False:
    mse_loss, r2_score = active_learning.TrainModel(rounds = 2, epochs = e, is_validation = False)

    # if is_validation is True:
    #mse_loss = active_learning.TrainModel(rounds = 2, epochs = e, is_validation = True)

    # save the trained model
    active_learning.saveModel(model = active_learning.model, optimizer = active_learning.optimizer, path_to_save = 'trainedModels/trained_weights')
    model, path = active_learning.getTrainedModel(rounds = 2)

active_learning.TestModel(rounds = 2)
    


running model
before loop, this is the train loader: <torch.utils.data.dataloader.DataLoader object at 0x15b4fe3d0> 0
epochs: 0, train loss: nan, train likelihood: nan, train kl: nan, train_avg_R2: nan
running model
before loop, this is the train loader: <torch.utils.data.dataloader.DataLoader object at 0x15b4fcc10> 1
running loop
epochs: 1, train loss: 1317.9713134765625, train likelihood: 1.2902977466583252, train kl: 1316.6810302734375, train_avg_R2: -0.01632559299468994
running model
before loop, this is the train loader: <torch.utils.data.dataloader.DataLoader object at 0x15b4fcc10> 1
running loop
epochs: 2, train loss: 1317.732666015625, train likelihood: 1.2904441356658936, train kl: 1316.4422607421875, train_avg_R2: -0.016655325889587402
running model
before loop, this is the train loader: <torch.utils.data.dataloader.DataLoader object at 0x15b4fcc10> 1
running loop
epochs: 3, train loss: 1317.4940185546875, train likelihood: 1.2906041145324707, train kl: 1316.203369140625, tra

AttributeError: 'MSELoss' object has no attribute 'item'